In [1]:
import config
from bs4 import BeautifulSoup as BS
import requests
import mysql.connector
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import *
import time
import re
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import KNNBasic
from surprise.model_selection import GridSearchCV
from sklearn.metrics.pairwise import cosine_similarity
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE

In [4]:
df_stats_total = pd.read_csv('df_stats_merged.csv', index_col='team')

In [59]:
df_stats_total4 = pd.read_csv('df_stats_merged4.csv', index_col='team')

In [22]:
df_leagues = pd.read_csv('df_stats_leagues.csv', index_col=0)

In [23]:
df_leagues.columns

Index(['league', 'team'], dtype='object')

In [25]:
df_leagues.shape

(98, 2)

In [26]:
df_leagues.head()

,league,team
0,Bundesliga,Bayern Munich
1,Bundesliga,Borussia Dortmund
4,Bundesliga,RasenBallsport Leipzig
2,Bundesliga,Bayer Leverkusen
8,Bundesliga,Borussia M.Gladbach


In [48]:
df_leagues['team']

0              Bayern Munich
1          Borussia Dortmund
4     RasenBallsport Leipzig
2           Bayer Leverkusen
8        Borussia M.Gladbach
               ...          
14         Parma Calcio 1913
15                     Genoa
10                    Empoli
18                 Frosinone
19                    Chievo
Name: team, Length: 98, dtype: object

In [60]:
df_stats_total4.head(10)

,INT,CR,ACR,CCOP,CCSP,BLK,TKL,FC,FS,P,...,SV,AKS,PKC,CL,PUNCH,ALB,ATB,LMT,TOFF,BCS
team,,,,,,,,,,,,,,,,,,,,,
Bayern Munich,336,777,201,340,71,61,464,312,346,19414,...,48,11,6,344,3,1292,31,1,88,52
Borussia Dortmund,385,458,114,234,42,79,497,292,389,17828,...,90,3,5,628,13,981,36,1,64,51
RasenBallsport Leipzig,420,616,156,288,53,70,653,465,400,11421,...,95,17,6,668,6,864,29,3,87,38
Bayer Leverkusen,364,642,157,305,34,99,598,370,400,18250,...,92,15,6,642,9,1197,23,0,66,44
Borussia M.Gladbach,310,565,140,258,46,102,500,332,365,14845,...,115,3,3,588,4,1067,19,2,70,30
Wolfsburg,430,681,176,226,48,95,540,358,445,12164,...,110,5,6,739,17,1449,16,4,67,37
Eintracht Frankfurt,380,686,155,210,55,103,612,432,353,9861,...,127,13,1,669,14,1094,20,1,84,45
Werder Bremen,407,676,164,274,39,109,552,437,350,13366,...,108,5,5,773,14,1086,14,2,40,30
Hoffenheim,400,666,179,364,56,104,574,471,355,13450,...,123,7,7,603,8,952,17,3,66,42


In [61]:
cos_sim = cosine_similarity(df_stats_total4)

In [62]:
cos_sim

array([[1.        , 0.99954729, 0.99007663, ..., 0.99705863, 0.98655587,
        0.9899202 ],
       [0.99954729, 1.        , 0.99115565, ..., 0.99760918, 0.98756154,
        0.99096585],
       [0.99007663, 0.99115565, 1.        , ..., 0.99718181, 0.99900142,
        0.99908405],
       ...,
       [0.99705863, 0.99760918, 0.99718181, ..., 1.        , 0.99576252,
        0.99747256],
       [0.98655587, 0.98756154, 0.99900142, ..., 0.99576252, 1.        ,
        0.99934542],
       [0.9899202 , 0.99096585, 0.99908405, ..., 0.99747256, 0.99934542,
        1.        ]])

In [28]:
epl_array = (df_leagues['team'][df_leagues['league'] == 'EPL']).tolist()
ligue_1_array = (df_leagues['team'][df_leagues['league'] == 'Ligue_1']).tolist()
serie_a_array = (df_leagues['team'][df_leagues['league'] == 'Serie_A']).tolist()
la_liga_array = (df_leagues['team'][df_leagues['league'] == 'La_liga']).tolist()
bundesliga_array = (df_leagues['team'][df_leagues['league'] == 'Bundesliga']).tolist()

In [63]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df_stats_total4.index)
#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(team, cosine_sim = cos_sim):
    
    epl_array = (df_leagues['team'][df_leagues['league'] == 'EPL']).tolist()
    ligue_1_array = (df_leagues['team'][df_leagues['league'] == 'Ligue_1']).tolist()
    serie_a_array = (df_leagues['team'][df_leagues['league'] == 'Serie_A']).tolist()
    la_liga_array = (df_leagues['team'][df_leagues['league'] == 'La_liga']).tolist()
    bundesliga_array = (df_leagues['team'][df_leagues['league'] == 'Bundesliga']).tolist()
    
    en_teams = []
    fr_teams = []
    it_teams = []
    sp_teams = []
    ge_teams = []
    
    # initializing the empty list of recommended movies
    recommended_teams = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == team].index[0]
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    # getting the indexes of the 10 most similar movies
    all_indexes = list(score_series.index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in all_indexes:
        recommended_teams.append(list(df_stats_total4.index)[i])
    
    if recommended_teams[0] in epl_array:
        for i in recommended_teams:
            if i in ligue_1_array:
                fr_teams.append(i)
            if i in serie_a_array:
                it_teams.append(i)
            if i in la_liga_array:
                sp_teams.append(i)
            if i in bundesliga_array:
                ge_teams.append(i)
            
        print(fr_teams[0], it_teams[0], sp_teams[0], ge_teams[0])
    
    elif recommended_teams[0] in ligue_1_array:
        for i in recommended_teams:
            if i in epl_array:
                en_teams.append(i)
            if i in serie_a_array:
                it_teams.append(i)
            if i in la_liga_array:
                sp_teams.append(i)
            if i in bundesliga_array:
                ge_teams.append(i)
        print(en_teams[0], it_teams[0], sp_teams[0], ge_teams[0])
    
    elif recommended_teams[0] in serie_a_array:
        for i in recommended_teams:
            if i in epl_array:
                en_teams.append(i)
            if i in ligue_1_array:
                fr_teams.append(i)
            if i in la_liga_array:
                sp_teams.append(i)
            if i in bundesliga_array:
                ge_teams.append(i)
        print(en_teams[0], fr_teams[0], sp_teams[0], ge_teams[0])
    
    elif recommended_teams[0] in la_liga_array:
        for i in recommended_teams:
            if i in epl_array:
                en_teams.append(i)
            if i in serie_a_array:
                it_teams.append(i)
            if i in ligue_1_array:
                fr_teams.append(i)
            if i in bundesliga_array:
                ge_teams.append(i)
        print(en_teams[0], fr_teams[0], it_teams[0], ge_teams[0])
        
    elif recommended_teams[0] in bundesliga_array:
        for i in recommended_teams:
            if i in epl_array:
                en_teams.append(i)
            if i in serie_a_array:
                it_teams.append(i)
            if i in la_liga_array:
                sp_teams.append(i)
            if i in ligue_1_array:
                fr_teams.append(i)
        print(en_teams[0], fr_teams[0], it_teams[0], sp_teams[0])
    
    else:
        print('not applicable')
    
    return 


In [75]:
recommendations('Nimes')

Brighton Genoa SD Huesca Eintracht Frankfurt


In [65]:
for team in df_leagues['team']:
    print(team , recommendations(team), '++++++++')

Liverpool Paris Saint Germain Napoli Real Madrid
Bayern Munich None ++++++++
Chelsea Paris Saint Germain Napoli Real Betis
Borussia Dortmund None ++++++++
Everton Nantes Fiorentina Girona
RasenBallsport Leipzig None ++++++++
Tottenham Lyon Napoli Real Betis
Bayer Leverkusen None ++++++++
Tottenham Lyon AC Milan Real Madrid
Borussia M.Gladbach None ++++++++
Wolverhampton Wanderers Strasbourg Cagliari Villarreal
Wolfsburg None ++++++++
Brighton Guingamp Genoa Leganes
Eintracht Frankfurt None ++++++++
Manchester United Saint-Etienne Lazio Celta Vigo
Werder Bremen None ++++++++
Manchester United Saint-Etienne Lazio Celta Vigo
Hoffenheim None ++++++++
Wolverhampton Wanderers Montpellier Chievo Girona
Fortuna Duesseldorf None ++++++++
Leicester Bordeaux Cagliari Villarreal
Hertha Berlin None ++++++++
Huddersfield Montpellier Torino Girona
Mainz 05 None ++++++++
Wolverhampton Wanderers Dijon Chievo Villarreal
Freiburg None ++++++++
Crystal Palace Nantes Torino Rayo Vallecano
Schalke 04 None +

In [87]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df_stats_total4.index)
#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def opposites(team, cosine_sim = cos_sim):
    
    epl_array = (df_leagues['team'][df_leagues['league'] == 'EPL']).tolist()
    ligue_1_array = (df_leagues['team'][df_leagues['league'] == 'Ligue_1']).tolist()
    serie_a_array = (df_leagues['team'][df_leagues['league'] == 'Serie_A']).tolist()
    la_liga_array = (df_leagues['team'][df_leagues['league'] == 'La_liga']).tolist()
    bundesliga_array = (df_leagues['team'][df_leagues['league'] == 'Bundesliga']).tolist()
    
    en_teams = []
    fr_teams = []
    it_teams = []
    sp_teams = []
    ge_teams = []
    
    # initializing the empty list of recommended movies
    recommended_teams = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == team].index[0]
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    # getting the indexes of the 10 most similar movies
    all_indexes = list(score_series.index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in all_indexes:
        recommended_teams.append(list(df_stats_total4.index)[i])
    
    if recommended_teams[0] in epl_array:
        for i in recommended_teams:
            if i in ligue_1_array:
                fr_teams.append(i)
            if i in serie_a_array:
                it_teams.append(i)
            if i in la_liga_array:
                sp_teams.append(i)
            if i in bundesliga_array:
                ge_teams.append(i)
            
        print(fr_teams[-5:], it_teams[-5:], sp_teams[-5:], ge_teams[-5:])
    
    elif recommended_teams[0] in ligue_1_array:
        for i in recommended_teams:
            if i in epl_array:
                en_teams.append(i)
            if i in serie_a_array:
                it_teams.append(i)
            if i in la_liga_array:
                sp_teams.append(i)
            if i in bundesliga_array:
                ge_teams.append(i)
        print(en_teams[-1], it_teams[-1], sp_teams[-1], ge_teams[-1])
    
    elif recommended_teams[0] in serie_a_array:
        for i in recommended_teams:
            if i in epl_array:
                en_teams.append(i)
            if i in ligue_1_array:
                fr_teams.append(i)
            if i in la_liga_array:
                sp_teams.append(i)
            if i in bundesliga_array:
                ge_teams.append(i)
        print(en_teams[-1], fr_teams[-1], sp_teams[-1], ge_teams[-1])
    
    elif recommended_teams[0] in la_liga_array:
        for i in recommended_teams:
            if i in epl_array:
                en_teams.append(i)
            if i in serie_a_array:
                it_teams.append(i)
            if i in ligue_1_array:
                fr_teams.append(i)
            if i in bundesliga_array:
                ge_teams.append(i)
        print(en_teams[-1], fr_teams[-1], it_teams[-1], ge_teams[-1])
        
    elif recommended_teams[0] in bundesliga_array:
        for i in recommended_teams:
            if i in epl_array:
                en_teams.append(i)
            if i in serie_a_array:
                it_teams.append(i)
            if i in la_liga_array:
                sp_teams.append(i)
            if i in ligue_1_array:
                fr_teams.append(i)
        print(en_teams[-1], fr_teams[-1], it_teams[-1], sp_teams[-1])
    
    else:
        print('not applicable')
    
    return 

In [91]:
opposites('Manchester City')

['Amiens', 'Reims', 'Guingamp', 'Caen', 'Nimes'] ['Torino', 'Bologna', 'Udinese', 'Genoa', 'Parma Calcio 1913'] ['Eibar', 'Leganes', 'SD Huesca', 'Alaves', 'Getafe'] ['Nuernberg', 'Augsburg', 'VfB Stuttgart', 'Schalke 04', 'Eintracht Frankfurt']


In [84]:
for team in df_leagues['team']:
    print(team , opposites(team), '++++++++')

Cardiff Nimes Parma Calcio 1913 Getafe
Bayern Munich None ++++++++
Cardiff Nimes Parma Calcio 1913 Getafe
Borussia Dortmund None ++++++++
Cardiff Paris Saint Germain Napoli Getafe
RasenBallsport Leipzig None ++++++++
Cardiff Nimes Parma Calcio 1913 Getafe
Bayer Leverkusen None ++++++++
Cardiff Nimes Parma Calcio 1913 Getafe
Borussia M.Gladbach None ++++++++
Cardiff Paris Saint Germain Napoli Getafe
Wolfsburg None ++++++++
Manchester City Paris Saint Germain Napoli Barcelona
Eintracht Frankfurt None ++++++++
Cardiff Nimes Parma Calcio 1913 Getafe
Werder Bremen None ++++++++
Cardiff Nimes Parma Calcio 1913 Getafe
Hoffenheim None ++++++++
Cardiff Paris Saint Germain Napoli Getafe
Fortuna Duesseldorf None ++++++++
Cardiff Paris Saint Germain Napoli Getafe
Hertha Berlin None ++++++++
Cardiff Paris Saint Germain Napoli Getafe
Mainz 05 None ++++++++
Cardiff Paris Saint Germain Napoli Getafe
Freiburg None ++++++++
Cardiff Paris Saint Germain Napoli Getafe
Schalke 04 None ++++++++
Cardiff Paris

In [94]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df_stats_total4.index)
#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def l_recommendations(team, cosine_sim = cos_sim):
    
    epl_array = (df_leagues['team'][df_leagues['league'] == 'EPL']).tolist()
    ligue_1_array = (df_leagues['team'][df_leagues['league'] == 'Ligue_1']).tolist()
    serie_a_array = (df_leagues['team'][df_leagues['league'] == 'Serie_A']).tolist()
    la_liga_array = (df_leagues['team'][df_leagues['league'] == 'La_liga']).tolist()
    bundesliga_array = (df_leagues['team'][df_leagues['league'] == 'Bundesliga']).tolist()
    
    en_teams = []
    fr_teams = []
    it_teams = []
    sp_teams = []
    ge_teams = []
    
    # initializing the empty list of recommended movies
    recommended_teams = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == team].index[0]
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    # getting the indexes of the 10 most similar movies
    all_indexes = list(score_series.index)
    
    print(score_series)
    
    # populating the list with the titles of the best 10 matching movies
#     for i in all_indexes:
#         recommended_teams.append(list(df_stats_total4.index)[i])
    
#     if recommended_teams[0] in epl_array:
#         for i in recommended_teams:
#             if i in ligue_1_array:
#                 fr_teams.append(i)
#             if i in serie_a_array:
#                 it_teams.append(i)
#             if i in la_liga_array:
#                 sp_teams.append(i)
#             if i in bundesliga_array:
#                 ge_teams.append(i)
            
#         print(fr_teams[0], it_teams[0], sp_teams[0], ge_teams[0])
    
#     elif recommended_teams[0] in ligue_1_array:
#         for i in recommended_teams:
#             if i in epl_array:
#                 en_teams.append(i)
#             if i in serie_a_array:
#                 it_teams.append(i)
#             if i in la_liga_array:
#                 sp_teams.append(i)
#             if i in bundesliga_array:
#                 ge_teams.append(i)
#         print(en_teams[0], it_teams[0], sp_teams[0], ge_teams[0])
    
#     elif recommended_teams[0] in serie_a_array:
#         for i in recommended_teams:
#             if i in epl_array:
#                 en_teams.append(i)
#             if i in ligue_1_array:
#                 fr_teams.append(i)
#             if i in la_liga_array:
#                 sp_teams.append(i)
#             if i in bundesliga_array:
#                 ge_teams.append(i)
#         print(en_teams[0], fr_teams[0], sp_teams[0], ge_teams[0])
    
#     elif recommended_teams[0] in la_liga_array:
#         for i in recommended_teams:
#             if i in epl_array:
#                 en_teams.append(i)
#             if i in serie_a_array:
#                 it_teams.append(i)
#             if i in ligue_1_array:
#                 fr_teams.append(i)
#             if i in bundesliga_array:
#                 ge_teams.append(i)
#         print(en_teams[0], fr_teams[0], it_teams[0], ge_teams[0])
        
#     elif recommended_teams[0] in bundesliga_array:
#         for i in recommended_teams:
#             if i in epl_array:
#                 en_teams.append(i)
#             if i in serie_a_array:
#                 it_teams.append(i)
#             if i in la_liga_array:
#                 sp_teams.append(i)
#             if i in ligue_1_array:
#                 fr_teams.append(i)
#         print(en_teams[0], fr_teams[0], it_teams[0], sp_teams[0])
    
#     else:
#         print('not applicable')
    
    return 

In [95]:
l_recommendations('Liverpool')

19    1.000000
3     0.999875
20    0.999843
1     0.999830
79    0.999802
        ...   
56    0.979645
32    0.974603
47    0.970225
43    0.946860
35    0.937408
Length: 98, dtype: float64


In [99]:
df_stats_total4.iloc[35]

INT       533
CR        564
ACR       148
CCOP      218
CCSP       56
BLK       147
TKL       655
FC        374
FS        394
P        6530
AW        978
BR       2265
DR        259
DW       2267
IBS       252
OBS       166
DSP       332
PK          4
FKCR       61
CRN       168
FKS        13
SV        141
AKS        17
PKC         7
CL        980
PUNCH      11
ALB       873
ATB        10
LMT         3
TOFF       77
BCS        19
Name: Cardiff, dtype: int64